In [1]:
from visual_networks import *
from sample_networks import *
import torch


# Generate some random data
x = torch.randn(50, 10)

# Initialize and run the model
model = SimpleLinearModel()
y_pred = model(x)

# Visualize the model
#visualize_model(model)

params = extract_params(model)


In [2]:
iterator = iter(params.items())
weights = []
biases = []
for first_item, second_item in zip(iterator, iterator):
    # Process the items here
    weights.append(first_item[1])
    biases.append(second_item[1])

In [3]:
from visual import *
network = Network(weights, biases)  # Example: 3 layers with 3, 4, and 2 neurons respectively
thread = Thread(target=run_glfw_window, args=(network,))
thread.start()

(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(655.0, 371.0)
(599.0, 335.0)
(599.0, 335.0)
(599.0, 335.0)
(599.0, 335.0)
(599.0, 335.0)
(599.0, 335.0)
(599.0, 335.0)
(599.0, 335.0)
(517.0, 387.0)
(517.0, 387.0)
(517.0, 387.0)
(517.0, 387.0)
(517.0, 387.0)
(517.0, 387.0)
(517.0, 387.0)
(517.0, 387.0)
(517.0, 387.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(511.0, 340.0)
(568.0, 322.0)
(568.0, 32

In [4]:
from tracking_objects import *
tracking_object()

FPS: 63
FPS: 60
0.89125 0.99
0.89125 0.9816666666666667
0.88375 0.805
0.88375 0.795
0.8775 0.6266666666666667
0.87875 0.495
0.88 0.4866666666666667
0.88875 0.39666666666666667
0.89 0.39166666666666666
0.90125 0.33
FPS: 60
0.91125 0.28833333333333333
0.91125 0.2866666666666667
0.92125 0.25666666666666665
0.935 0.23333333333333334
0.95375 0.21333333333333335
0.95625 0.21
0.97625 0.19
0.9775 0.18833333333333332
0.9975 0.16833333333333333
0.995 0.041666666666666664
0.99375 0.041666666666666664
0.98625 0.041666666666666664
0.985 0.041666666666666664
0.97875 0.041666666666666664
0.9775 0.04
0.97375 0.03166666666666667
0.97 0.023333333333333334
0.96625 0.015
0.96375 0.006666666666666667
0.96 0.0016666666666666668


c:\ProgramData\Miniconda3\lib\site-packages\glfw\__init__.py:916: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
